In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


**City of New York 2021 Democratic Mayor**


---

RCV Winner: Eric L. Adams after 8 rounds


---

https://www.vote.nyc/sites/default/files/pdf/election_results/2021/20210622Primary%20Election/rcv/DEM%20Mayor%20Citywide.pdf

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict


data = pd.read_csv('/content/drive/My Drive/Summer Research 24/NycMayorVote/aggregated_voting_NYCMayor.csv')
data

,1st,2nd,3rd,4th,5th,count
0,217572,Null,Null,Null,Null,64274
1,217796,Null,Null,Null,Null,20566
2,219469,Null,Null,Null,Null,12771
3,219978,Null,Null,Null,Null,10258
4,217572,219469,Null,Null,Null,9833
...,...,...,...,...,...,...
62149,217796,218117,219469,217605,218922,1
62150,219469,218117,221183,217605,218922,1
62151,217796,218117,219469,217605,Null,1
62152,219469,218117,221183,217605,217654,1


In [3]:
nameTable = pd.read_csv('/content/drive/My Drive/Summer Research 24/NycMayorVote/NYCCandidates.csv')

def id_to_name(candidacy_id):
  if candidacy_id in nameTable['CandidacyID'].values:
        name = nameTable.loc[nameTable['CandidacyID'] == candidacy_id, 'DefaultBallotName'].values[0]
        return name
  else:
        return None

nameTable

,CandidacyID,DefaultBallotName
0,217561,Arthur Z. Schwartz
1,217562,Jumaane D. Williams
2,217563,Christopher A. Sosa
3,217564,Lindsey C. Boylan
4,217566,Fernando A. Aquino
...,...,...
801,221632,Justice Z. Cruz
802,221634,Hei Man Chan
803,221635,Jeanine Williams
804,221643,Bernadette McNear


**Plurality Method:** Eric Adams

In [4]:
# Count the number of 1st place votes each item gets, weighted by 'count' column
first_place_counts = data.groupby('1st')['count'].sum()

# Get the ID with the highest count
highest_id = first_place_counts.idxmax()

# Use that ID to get the name from nametable
# name = nameTable.loc[nameTable['CandidacyID'] == highest_id, 'DefaultBallotName'].values[0]

print(f"Winner ID: {highest_id}")
print(f"Winner: {id_to_name(int(highest_id))}")

Winner ID: 217572
Winner: Eric L. Adams


**Borda Count:** Eric Adams

In [5]:
def borda_count(df):
    points = defaultdict(int)
    for _, row in df.iterrows():
        if pd.notna(row['1st']):
            points[row['1st']] += 4 * row['count']
        if pd.notna(row['2nd']):
            points[row['2nd']] += 3 * row['count']
        if pd.notna(row['3rd']):
            points[row['3rd']] += 2 * row['count']
        if pd.notna(row['4th']):
            points[row['4th']] += 1 * row['count']

    borda_scores = dict(points)
    winner = max(borda_scores, key=borda_scores.get)

    return winner, borda_scores

winner, scores = borda_count(data)
print(f"Winner: {id_to_name(int(winner))}")
print(f"Borda Scores: {scores}")

Winner: Eric L. Adams
Borda Scores: {'217572': 1638035, 'Null': 1255518, '217796': 978878, '219469': 1448015, '219978': 1454984, '217605': 760707, '218491': 476634, '218127': 452127, '221183': 515183, '221141': 160522, '217654': 67231, '218117': 84697, 'Write-in': 13355, '218922': 92150, '221458': 52224}


In [6]:
data.replace("Null", np.nan, inplace=True)


def copeland_winner(df):
    # Extract candidates from all rank columns
    candidates = set(df[['1st', '2nd', '3rd', '4th', '5th']].stack().dropna().unique())
    pairwise_wins = defaultdict(int)
    pairwise_losses = defaultdict(int)
    #print(len(candidates))
    #print(candidates)

    for candidate1 in candidates:
        #print("C1 ", candidate1)
        for candidate2 in candidates:
            # print("C2 ", candidate2)
            if candidate1 != candidate2:
                wins = 0
                losses = 0
                for _, row in df.iterrows():
                    ranks = row[['1st', '2nd', '3rd', '4th', '5th']].tolist()
                    if candidate1 in ranks:
                        rank1 = ranks.index(candidate1)
                    else:
                        rank1 = len(candidates)
                    if candidate2 in ranks:
                        rank2 = ranks.index(candidate2)
                    else:
                        rank2 = len(candidates)
                    if rank1 < rank2:
                        wins += row['count']
                    elif rank1 > rank2:
                        losses += row['count']
                if wins > losses:
                    pairwise_wins[candidate1] += 1
                    pairwise_losses[candidate2] += 1
                # elif losses > wins:
                #     pairwise_wins[candidate2] += 1
                #     pairwise_losses[candidate1] += 1

    copeland_scores = {candidate: pairwise_wins[candidate] for candidate in candidates}
    winner = max(copeland_scores, key=copeland_scores.get)

    return winner, copeland_scores

winner, scores = copeland_winner(data)
print(f"Winner: {id_to_name(int(winner))}")
print(f"Copeland Scores: {scores}")

Winner: Eric L. Adams
Copeland Scores: {'218922': 4, '221141': 5, '218491': 7, '219978': 11, 'Write-in': 0, '218117': 3, '217572': 13, '217796': 10, '218127': 6, '219469': 12, '221458': 2, '217654': 1, '221183': 8, '217605': 9}
